# Photosynthesis estimation function

## Goal

**Estimate the photosynthetic output of an average *Synechocystis* sp. PCC 6803 cell as input into another model**

## Plan

First option: Use the submitted model version. The model is unalterend (not reduced in size) and the cellular chlorophyll a concentration is only considered for normalizing the pigment concentrations.

### Input
- **Light intensity & spectrum (400-700 nm)**
- Pigment concentrations: **Chlorophyll A, beta-carotene (carotenoids), phycocyanin, and allophycocyanin**
- Use the **measured absorption spectrum to calculate the light attenuation**
- NOT Chlorophyll concentration in sample cuvette BUT **Cell density in cells/ml**

### Output

NOT Concentrations BUT **Rates of ATP, NADPH, 3PGA, Fd_red**

## Predict the rates

In [1]:
import numpy as np
import pandas as pd
from modelbase.ode import Model, Simulator
from modelbase.ode import ratefunctions as rf

import sys
sys.path.append("../Code")
import functions_light_absorption as lip
from function_predict_simulated_ssrates_biomass import get_simulated_ssrates, get_influx_rate_estimations
from parameters import p

In [ ]:
# Get the rates under 100 µE of solar light with default model pigment content 
# and assuming we want 10% of the maximal ATP, 50% maximal NADPH, 10 maximal Fd_red and 30% maximal 3PGA production
rates = get_simulated_ssrates(
    lip.light_spectra("solar", 100),
    pigment_content=p["pigment_content"],
)
rates

## Interface between Andreas model and the prediction function

In [ ]:
# Example output
# The rows contain the flux of a certain production reaction
# The columns are the fluxes of all metabolites determined for a model where the compound in the name was drained via a sink reaction.
# Other meatbolites were only consumed if the metabolite pool was filled to over 95% (analogous to synechocystis ETC paper)
rates = get_influx_rate_estimations(
        light_intensity=100, # Model
        cell_density=1e8, # [cells ml^-1]
        chlorophyll=np.array([10000]), # [µmol l^-1]
        carotenoids=np.array([3000]), # [µmol l^-1]
        phycocyanin=None, # [µmol l^-1]
        allophycocyanin=None, # [µmol l^-1]
        beta_carotene_fraction=0.26, # [rel] fraction of beta-carotene of cellular carotenoids
        ps_ratio=5.9,
        sample_depth_m=0.01, # [m] Assuming a cuvette with 1 cm diameter
    )
rates

## Mock model to add the funciton dynamically
The function calculates the steady-state input rates at every time point.
For that, I calculate the fraction of each metabolite pool the this not filled (e.g. ADP in the ATP pool).
Then I use the relative size of these "used-up" metabolite pools as the ratios that are the input to the rate estimation function.
The fractions are used to scale the maximal steady-state produciton rates for each metabolite that I simulated under the given conditions.

The pigment contents are currently assumed fixed and a few tweaks have to be made to the funcitons to make them dynamic.

In [20]:
# Create a mock model
m = Model()
# Add the compounds
m.add_compounds(["ATP", "NADPH", "3PGA", "Fd_red"])

# Add total pools or maximal concentrations
totals = {"ATP":"ATP_tot", "NADPH":"NADP_tot", "3PGA":"3PGA_max", "Fd_red":"Fd_tot"}

p = {
    "k_ATP": 5,
    "k_NADPH": 5,
    "k_3PGA": 5,
    "k_Fd_red": 5,
    "ATP_tot": 1,
    "NADP_tot": 1,
    "3PGA_max": 1,
    "Fd_tot": 1,

    "light":100, # [µmol(Photons) m^-2 s^-1] Light intensity (warm white LED) 
    "cell_density":1e8, # [cells ml^-1]
    "chlorophyll":10000, # [µmol l^-1]
    "carotenoids":3000, # [µmol l^-1]
    "phycocyanin":None, # [µmol l^-1]
    "allophycocyanin":None, # [µmol l^-1]
    "beta_carotene_fraction":0.26, # [rel] fraction of beta-carotene of cellular carotenoids
    "ps_ratio":5.9,
    "sample_depth_m":0.01, # [m] Assuming a cuvette with 1 cm diameter
    "cell_volume": 4e-15,
}
m.add_parameters(p)

def fraction(S, S_tot):
    return S/S_tot

# Add algebraic models
for comp in ["ATP", "NADPH", "3PGA", "Fd_red"]:
    m.add_algebraic_module_from_args(
        f"{comp} fraction",
        function=fraction,
        derived_compounds=[f"{comp}_fraction"],
        args=[comp, totals[comp]]
    )

    m.add_reaction_from_args(
        f"{comp} consumption",
        function=rf.mass_action_1,
        stoichiometry={comp:-1},
        args=[f"k_{comp}", comp]
    )

In [ ]:
s=Simulator(m)
s.initialise({"ATP":1, "NADPH":0.9, "3PGA":0.8, "Fd_red":0.7})
s.simulate(1)
s.plot()

### Wrapper function

### Add production to mock model

In [22]:
# Estimate the required production ratios by the fill state of the metabolite pools
def get_required_production_ratio(*args):
    frac = np.stack(args, axis=0)
    frac = 1 - frac
    rel_frac = frac / frac.sum(axis=0)
    return rel_frac

m.add_algebraic_module_from_args(
    "production ratios",
    function=get_required_production_ratio,
    derived_compounds=[f"{comp}_prodratio" for comp in ["ATP", "NADPH", "3PGA", "Fd_red"]],
    args=[f"{comp}_fraction" for comp in ["ATP", "NADPH", "3PGA", "Fd_red"]]
)

m.add_algebraic_module_from_args(
    "production_rates",
    function=get_influx_rate_estimations_modelbase,
    derived_compounds=["vATP_prod", "vNADPH_prod", "v3PGA_prod", "vFd_red_prod"],
    args=[
        "light",
        "ATP_prodratio",
        "NADPH_prodratio",
        "3PGA_prodratio",
        "Fd_red_prodratio",
        "cell_density",
        "chlorophyll",
        "carotenoids",
        "phycocyanin",
        "allophycocyanin",
        "ps_ratio",
        "beta_carotene_fraction",
        "sample_depth_m",
        "cell_volume",
    ]
)

# Add algebraic models
for comp in ["ATP", "NADPH", "3PGA", "Fd_red"]:
    m.add_reaction_from_args(
        f"{comp} production",
        function=rf.constant,
        stoichiometry={comp:1},
        args=[f"v{comp}_prod"]
    )

In [ ]:
s=Simulator(m)
s.initialise({"ATP":1, "NADPH":0.9, "3PGA":0.8, "Fd_red":0.7})
s.simulate(10)
s.plot()

In [ ]:
s.get_full_results_df()